## Viewing PowerSimulations as Optimization Model

This is a jupyter notebook for loading 5 bus simulation. The goal is to visualize it as a JuMP so we can later manipulate constraints.

First we'll load packages and load files.

In [1]:
using Pkg
Pkg.activate("..")

  Activating project at `/lustre/eaglefs/projects/pvb/cju/ProgressiveHedging.jl`


In [2]:
using PowerSystems
using PowerSimulations
using Dates
using Logging
using DataFrames
logger = configure_logging(console_level = Logging.Info)
const PSI = PowerSimulations
const PSY = PowerSystems
using DrWatson

In [3]:
using JuMP
using Xpress
solver = optimizer_with_attributes(Xpress.Optimizer, "MIPRELSTOP" => 1e-5, "OUTPUTLOG" => 1, "PRESOLVE" => 0,  "MAXTIME" => 300)
#using Cbc
#solver = optimizer_with_attributes(Cbc.Optimizer, "MIPRELSTOP" => 1e-5, "OUTPUTLOG" => 1, "PRESOLVE" => 0, "MAXTIME" => 300)

[ Info: Xpress: Found license file /nopt/nrel/apps/xpressmp/8.13.0/bin/xpauth.xpr
[ Info: Xpress: Development license detected.


MathOptInterface.OptimizerWithAttributes(Xpress.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("MIPRELSTOP") => 1.0e-5, MathOptInterface.RawOptimizerAttribute("OUTPUTLOG") => 1, MathOptInterface.RawOptimizerAttribute("PRESOLVE") => 0, MathOptInterface.RawOptimizerAttribute("MAXTIME") => 300])

In [4]:
### Parsing Args
sys_name = "/lustre/eaglefs/projects/pvb/5_bus_battery_testing/data/new_systems/pv=30_storagehours=10_wind=30/sys.json"
interval = 24
horizon = 168
steps = 3
battery = true

params = (
    originalsim = sys_name, # replace(split(sys_name, "/")[end-1], "=" => "-"),
    interval = interval,
    horizon = horizon,
    steps = steps,
    battery = battery
)
println(params)
name = savename(params)

(originalsim = "/lustre/eaglefs/projects/pvb/5_bus_battery_testing/data/new_systems/pv=30_storagehours=10_wind=30/sys.json", interval = 24, horizon = 168, steps = 3, battery = true)


"battery=true_horizon=168_interval=24_originalsim=/lustre/eaglefs/projects/pvb/5_bus_battery_testing/data/new_systems/pv=30_storagehours=10_wind=30/sys.json_steps=3"

First, we will create the PowerSystem.

In [5]:
template_uc = template_unit_commitment(network = NetworkModel(DCPPowerModel, duals = [NodalBalanceActiveConstraint], use_slacks = true))
set_device_model!(template_uc, ThermalStandard, ThermalStandardUnitCommitment)

set_device_model!(template_uc, DeviceModel(GenericBattery, BookKeeping; duals=[EnergyBalanceConstraint]))

set_device_model!(template_uc, MonitoredLine, StaticBranchBounds)
set_device_model!(template_uc, Line, StaticBranch)
set_device_model!(template_uc, Transformer2W, StaticBranch)
set_device_model!(template_uc, TapTransformer, StaticBranch)
set_device_model!(template_uc, HVDCLine, HVDCLossless)

sys = System(datadir(sys_name); time_series_directory = "/tmp/scratch")
PSY.transform_single_time_series!(sys, horizon, Hour(interval))

┌ Warning: Overwriting ThermalStandard existing model
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/HKwUO/src/core/device_model.jl:84
┌ Warning: Overwriting Line existing model
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/HKwUO/src/core/device_model.jl:84
┌ Warning: Overwriting Transformer2W existing model
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/HKwUO/src/core/device_model.jl:84
┌ Warning: Overwriting TapTransformer existing model
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/HKwUO/src/core/device_model.jl:84
┌ Warning: Overwriting HVDCLine existing model
└ @ PowerSimulations ~/.julia/packages/PowerSimulations/HKwUO/src/core/device_model.jl:84
[ Info: Loaded time series from storage file existing=sys_time_series_storage.h5 new=/tmp/scratch/jl_CiF15z compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)
┌ Warning: Invalid range
│   valid_info.struct_name = "Bus"
│   field_name = "magnitude"
│   valid_range = "voltage

Next, we will create the PowerSimulation, i.e., the optimization model.

In [6]:
models = SimulationModels(
    decision_models = [
        DecisionModel(
            template_uc,
            sys,
            name = "UC",
            optimizer = solver,
            optimizer_solve_log_print = true,
        ),
    ],
)

Model Name,Model Type,Status,Output Directory
UC,GenericOpProblem,EMPTY,nothing


### Explore the SimulationModels

In [7]:
md_dec = PowerSimulations.get_decision_models(models)[1]

Network Model
┌───────────────┬──────────────────────────────┐
│ Network Model │ DCPPowerModel                │
│ Slacks        │ true                         │
│ PTDF          │ false                        │
│ Duals         │ NodalBalanceActiveConstraint │
└───────────────┴──────────────────────────────┘

Device Models
┌──────────────────────┬───────────────────────────────┬────────┐
│ Device Type          │ Formulation                   │ Slacks │
├──────────────────────┼───────────────────────────────┼────────┤
│ ThermalStandard      │ ThermalStandardUnitCommitment │ false  │
│ HydroDispatch        │ HydroDispatchRunOfRiver       │ false  │
│ PowerLoad            │ StaticPowerLoad               │ false  │
│ RenewableFix         │ FixedOutput                   │ false  │
│ RenewableDispatch    │ RenewableFullDispatch         │ false  │
│ GenericBattery       │ BookKeeping                   │ false  │
│ HydroEnergyReservoir │ HydroDispatchRunOfRiver       │ false  │
│ InterruptibleLoad    │ InterruptiblePowerLoad        │ false  │
└──────────────────────┴───────────────────────────────┴────────┘

Branch Models
┌────────────────┬────────────────────┬────────┐
│ Branch Type    │ Formulation        │ Slacks │
├────────────────┼────────────────────┼────────┤
│ MonitoredLine  │ StaticBranchBounds │ false  │
│ Line           │ StaticBranch       │ false  │
│ TapTransformer │ StaticBranch       │ false  │
│ Transformer2W  │ StaticBranch       │ false  │
│ HVDCLine       │ HVDCLossless       │ false  │
└────────────────┴────────────────────┴────────┘

Service Models
┌──────────────────────────────┬──────────────┬────────┬──────────────────┐
│ Service Type                 │ Formulation  │ Slacks │ Aggregated Model │
├──────────────────────────────┼──────────────┼────────┼──────────────────┤
│ VariableReserve{ReserveUp}   │ RangeReserve │ false  │ true             │
│ VariableReserve{ReserveDown} │ RangeReserve │ false  │ true             │
└──────────────────────────────┴──────────────┴────────┴──────────────────┘

We can now link the PowerSystem with the PowerSimulation model.

In [8]:
sequence = SimulationSequence(
    models = models,
    ini_cond_chronology = InterProblemChronology()
)

sim = Simulation(
    name = name,
    steps = steps,
    models = models,
    sequence = sequence,
    simulation_folder = datadir("powersim_results")
)

Simulation
┌─────────────────┬─────────────────────────────────────────────────────────────
│ Simulation Name │ battery=true_horizon=168_interval=24_originalsim=/lustre/e ⋯
│ Build Status    │ EMPTY                                                      ⋯
│ Run Status      │ NOT_READY                                                  ⋯
│ Initial Time    │ Unset Initial Time                                         ⋯
│ Steps           │ 3                                                          ⋯
└─────────────────┴─────────────────────────────────────────────────────────────
                                                                1 column omitted

Decision Models
┌────────────┬──────────────────┬────────┬──────────────────┐
│ Model Name │ Model Type       │ Status │ Output Directory │
├────────────┼──────────────────┼────────┼──────────────────┤
│ UC         │ GenericOpProblem │ EMPTY  │ nothing          │
└────────────┴──────────────────┴────────┴──────────────────┘

No Emulator Model Specified

Simulation Sequence
┌──────────────────────────┬──────────┐
│ Simulation Step Interval │ 24 hours │
│ Number of Problems       │ 1        │
└──────────────────────────┴──────────┘

Simulation Problems
┌────────────┬─────────┬──────────────┬─────────────────────┐
│ Model Name │ Horizon │ Interval     │ Executions Per Step │
├────────────┼─────────┼──────────────┼─────────────────────┤
│ UC         │ 168     │ 1440 minutes │ 1                   │
└────────────┴─────────┴──────────────┴─────────────────────┘

In [9]:
methodswith(typeof(sim))

[1] SimulationResults(sim::Simulation; ignore_status, kwargs...) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/simulation/simulation_results.jl:110
[2] build!(sim::Simulation; output_dir, recorders, console_level, file_level, serialize, initialize_problem) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/simulation/simulation.jl:528
[3] execute!(sim::Simulation; kwargs...) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/simulation/simulation.jl:855
[4] get_name(sim::Simulation) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/simulation/simulation.jl:104
[5] get_simulation_model(s::Simulation, name) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/simulation/simulation.jl:84
[6] show(io::IO, ::MIME{Symbol("text/plain")}, input::Simulation) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/utils/printing.jl:373
[7] show(io::IO, ::MIME{Symbol("text/html")}, input::Simulation) in PowerSimulations at /home/cju/.julia/packages/PowerSimulations/HKwUO/src/utils/printing.jl:377

### Build the system!

This will take some time.

In [10]:
build!(sim)

┌ Info: 
│  ────────────────────────────────────────────────────────────────────────────────
│                                         Time                    Allocations      
│                                ───────────────────────   ────────────────────────
│        Tot / % measured:            62.7s /  99.1%           7.27GiB /  99.8%    
│ 
│  Section               ncalls     time    %tot     avg     alloc    %tot      avg
│  ────────────────────────────────────────────────────────────────────────────────
│  Build Simulation           1    62.2s  100.0%   62.2s   7.26GiB  100.0%  7.26GiB
│    Build Problems           1    49.6s   79.7%   49.6s   6.01GiB   82.8%  6.01GiB
│      Problem UC             1    48.4s   77.8%   48.4s   5.93GiB   81.7%  5.93GiB
│        Build pre-step       1    40.9s   65.8%   40.9s   5.33GiB   73.4%  5.33GiB
│          Build Init...      1    31.0s   49.9%   31.0s   4.05GiB   55.9%  4.05GiB
│            DCPPower...      1    11.5s   18.4%   11.5s   1.50G

BuildStatus.BUILT = 0

In [11]:
using Printf

In [14]:
# taken from _execute! in PowerSimulations
# also PowerSimulations.jl/src/core/optimization_container.jl
sequence = PowerSimulations.get_sequence(sim)
models = PowerSimulations.get_models(sim)
execution_order = PowerSimulations.get_execution_order(sim)

for (ix, model_number) in enumerate(execution_order)
    println("[", ix, "]")
    model = get_simulation_model(models, model_number)
    model_name = get_name(model)
    
    # println("Methods: ", InteractiveUtils.methodswith(typeof(model)))
    
    container = PowerSimulations.get_optimization_container(model)
    jump_model = PowerSimulations.get_jump_model(container)
    
    
    # println("Vars: ", fieldnames(typeof(model)))
    
    # println(typeof(jump_model))
    
    fname = @sprintf "model_%i.txt" ix
    println("Writing to ", fname)
    open(fname, "w") do io
        write(io, jump_model)
    end;
    # println(jump_model)
    
    # print(model)
    println("======")
end

[1]
Writing to model_1.txt


### Run the model

In [13]:
# execute!(sim)